In [1]:
%matplotlib inline
from IPython.display import clear_output

In [2]:
import os
import sys
import pickle
import random
from collections import defaultdict
from itertools import product
import networkx as nx
from networkx.algorithms.approximation import clique
from joblib import Parallel, delayed

import Orange
import matplotlib
from matplotlib.cm import coolwarm, Spectral_r
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, f1_score, label_ranking_average_precision_score
from sklearn.svm import SVR
from tqdm.notebook import tqdm

sys.path.append('../../pygkernels')
from pygkernels.data import Datasets

from sbm_neighbour_score import sbm_neighbour_score

In [3]:
kernels_names = [
    'Katz', 'logKatz',
    'For', 'logFor',
    'Comm', 'logComm',
    'Heat', 'logHeat',
    'NHeat', 'logNHeat',
    'SCT', 'SCCT',
    'RSP', 'FE',
    'PPR', 'logPPR',
    'ModifPPR', 'logModifPPR',
    'HeatPR', 'logHeatPR',
    'DF', 'logDF',
    'Abs', 'logAbs',
    'SP-CT'
]

shuffle = lambda x: sorted(x, key=lambda k: random.random())

def dict_argmax(dct, score_key):
    best_key = list(dct.keys())[0]
    best_val = dct[best_key]
    for k, v in dct.items():
        if v[score_key] > best_val[score_key]:
            best_key, best_val = k, v
    return best_key, best_val

CACHE_ROOT = '../../cache/cache'

def load_or_calc_and_save(filename, force_calc=False, ignore_if_exist=False):
    def decorator(func):
        def wrapped(*args, **kwargs):
            if os.path.exists(filename) and not force_calc:
                print(f'{func.__name__}: cache file {filename} found! Skip calculations')
                if not ignore_if_exist:
                    with open(filename, 'rb') as f:
                        result = pickle.load(f)
                else:
                    result = None
            else:
                print(f'{func.__name__}: RECALC {filename}.\nargs: {", ".join(args)}, kwargs: {", ".join([f"{k}={v}" for k, v in kwargs.items()])}')
                result = func(*args, **kwargs)
                with open(filename, 'wb') as f:
                    pickle.dump(result, f)
            return result
        return wrapped
    return decorator

def calc_avranks(results):  # {dataset: {classifier: accuracy}}
    ranks = defaultdict(list)
    for dataset, classifier_accuracy in results.items():
        if type(dataset) == tuple:
            dataset = '_'.join([str(x) for x in dataset])
        classifiers, accuracies = zip(*list(classifier_accuracy.items()))
        for classifier, rank in zip(classifiers, rankdata(accuracies)):
            ranks[classifier].append(rank)
    ranks = {k: np.mean(v) for k, v in sorted(ranks.items(), key=lambda x: x[0])}
    return list(ranks.values()), list(ranks.keys()), len(results)

def ytrue_to_partition(y_true):
    partition = defaultdict(list)
    for idx, class_ in enumerate(y_true):
        partition[class_].append(idx)
    return list(partition.values())

In [4]:
DATASETS_RESULTS_ROOT = '../../cache/kkmeans_init_datasets'
datasets = [
    'cora_DB', 'cora_EC', 'cora_HA', 'cora_HCI', 'cora_IR', 'cora_Net',
    'dolphins',
    'eu-core',
    'eurosis',
    'football',
    'karate',
    'news_2cl1_0.1', 'news_2cl2_0.1', 'news_2cl3_0.1',
    'news_3cl1_0.1', 'news_3cl2_0.1', 'news_3cl3_0.1',
    'news_5cl1_0.1', 'news_5cl2_0.1', 'news_5cl3_0.1',
    'polblogs',
    'polbooks',
    'sp_school_day_1', 'sp_school_day_2'
]

In [5]:
with open(f'{CACHE_ROOT}/datasets_inits_bestparam_byari_individual_0.1.pkl', 'rb') as f:
    results = pickle.load(f)
with open(f'{CACHE_ROOT}/datasets_modularity_0.1.pkl', 'rb') as f:
    modularity_results = pickle.load(f)
    
for key in list(results.keys()):
    if key[0] not in datasets:
        del results[key]
        
for key in list(results.keys()):
    if key[0] not in datasets:
        del modularity_results[key]

In [6]:
results_modularity_any3 = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))  # {dataset: {graphidx: {kernel_name: best_ari}}}
for (dataset, kernel_name, graph_idx), si_ari in results.items():
    results_modularity_any3[dataset][graph_idx][kernel_name] = si_ari['modularity_any3']

In [7]:
def extract_feature(dataset_name, feature, G, partition, sp, max_clique):
    # graph-independent features
    n, k, p_in, p_out = dataset_name
    if feature == 'n':
        return n
    elif feature == 'k':
        return k
    elif feature == 'p_in':
        return p_in
    elif feature == 'p_out':
        return p_out
    elif feature == 'n/k':
        return n / k
    elif feature == 'p_in/p_out':
        return p_in / p_out
    
    elif feature == 'log(n)':
        return n
    elif feature == 'log(k)':
        return k
    elif feature == 'log(p_in)':
        return p_in
    elif feature == 'log(p_out)':
        return p_out
    elif feature == 'log(n/k)':
        return n / k
    elif feature == 'log(p_in/p_out)':
        return p_in / p_out
    
    elif feature == 'n/k * p_in/p_out':
        return (n / k) * (p_in / p_out)
    elif feature == 'log(n)/k * p_in/p_out':
        return np.log(n) / k * (p_in / p_out)
    elif feature == 'log(n/k) * p_in/p_out':
        return np.log(n / k) * (p_in / p_out)
    elif feature == 'log(n/k * p_in/p_out)':
        return np.log((n / k) * (p_in / p_out))
    
    elif feature == 'sbm_neighbour_score':
        return sbm_neighbour_score(int(n), int(k), p_in, p_out)
    
    # graph-dependant features
    elif feature == 'modularity':
        return nx.community.modularity(G, partition)
    elif feature == 'diameter':
        return nx.diameter(G)
    elif feature == 'density':
        return nx.density(G)
    elif feature == 'avg_deg':
        return np.mean(G.degree)
    elif feature == 'std_deg':
        return np.std(G.degree)
    elif feature == 'avg(deg | deg > avg_deg)':
        deg = np.array(G.degree)
        return np.mean(deg[deg > np.mean(deg)])
    elif feature == 'median_deg':
        return np.median(G.degree)
    elif feature == 'max_deg':
        return np.max(G.degree)
    elif feature == 'avg_sp':
        return nx.average_shortest_path_length(G)
    elif feature == 'std_sp':
        return np.std(sp)
    elif feature == 'median_sp':
        return np.median(sp)
    elif feature == 'max_sp':
        return np.max(sp)
    elif feature == 'max_clique':
        return max_clique
    elif feature == 'max_clique/(n/k)':
        return max_clique/(n/k)
    else:
        raise Exception()

# Feature importance

In [8]:
feature_names = [
    'n', 'k', 'p_in', 'p_out', 'n/k', 'p_in/p_out',
    'log(n)/k * p_in/p_out', 'n/k * p_in/p_out', 'log(n/k) * p_in/p_out', 'log(n/k * p_in/p_out)',
    'sbm_neighbour_score',
    'modularity', 'diameter', 'density', 
    'avg_deg', 'std_deg', 'avg(deg | deg > avg_deg)', 'median_deg', 'max_deg',
    'avg_sp', 'std_sp', 'median_sp', 'max_sp', 
    'max_clique', 'max_clique/(n/k)'
]

In [9]:
def prepare_column(column):
    @load_or_calc_and_save(f'{CACHE_ROOT}/feature_importance/{column}.pkl')
    def wrapper():
        X, ya, yr = [], [], []
        (A, partition), info = Datasets()[column]
        n, k, p_in, p_out = info['n'], info['k'], info['p_in'], info['p_out']
        G = nx.from_numpy_matrix(A)
        partition = ytrue_to_partition(partition) 
        sp = [l for u in G for v, l in nx.single_source_shortest_path_length(G, u).items()]
        max_clique = len(clique.max_clique(G))
        features = [extract_feature((n, k, p_in, p_out), feature_name, G, partition, sp, max_clique) for feature_name in feature_names]
        for graph_idx in range(7):
            graph_ari = [v for k, v in sorted(list(results_modularity_any3[column][graph_idx].items()), key=lambda x: x[0])]
            graph_ranks = calc_avranks({0: results_modularity_any3[column][graph_idx]})[0]

            X.append(features)
            ya.append(graph_ari)
            yr.append(graph_ranks)
        return X, ya, yr
    
    return wrapper()
    
Xy_list = Parallel(n_jobs=1)(delayed(prepare_column)(column) for column in tqdm(results_modularity_any3.keys()))

X, y, X_train, y_train, X_val, y_val = [], [], [], [], [], []
for Xi, _, yi in Xy_list:
    X.extend(Xi)
    y.extend(yi > (np.max(yi, axis=1, keepdims=True) - 0.05))
    X_train.extend(Xi[:5])
    y_train.extend(yi[:5] > (np.max(yi[:5], axis=1, keepdims=True) - 0.05))
    X_val.extend(Xi[5:])
    y_val.extend(yi[5:] > (np.max(yi[5:], axis=1, keepdims=True) - 0.05))
    
X, y, X_train, y_train, X_val, y_val = np.array(X), np.array(y), np.array(X_train), np.array(y_train), np.array(X_val), np.array(y_val)

wrapper: cache file ../../cache/cache/feature_importance/cora_DB.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/cora_EC.pkl found! Skip calculations
wrapper: cache file ../../cache/cache/feature_importance/cora_HA.pkl found! Skip calculations
wrapper: RECALC ../../cache/cache/feature_importance/cora_HCI.pkl.
args: , kwargs: 
wrapper: cache file ../../cache/cache/feature_importance/cora_IR.pkl found! Skip calculations
wrapper: RECALC ../../cache/cache/feature_importance/cora_Net.pkl.
args: , kwargs: 
wrapper: RECALC ../../cache/cache/feature_importance/dolphins.pkl.
args: , kwargs: 
wrapper: RECALC ../../cache/cache/feature_importance/eu-core.pkl.
args: , kwargs: 
wrapper: RECALC ../../cache/cache/feature_importance/eurosis.pkl.
args: , kwargs: 
wrapper: RECALC ../../cache/cache/feature_importance/football.pkl.
args: , kwargs: 
wrapper: RECALC ../../cache/cache/feature_importance/karate.pkl.
args: , kwargs: 
wrapper: RECALC ../../cache/cache/featur

In [10]:
X_train[0]

array([1.00600000e+03, 7.00000000e+00, 2.28103279e-02, 2.95176881e-03,
       1.43714286e+02, 7.72768107e+00, 7.63245103e+00, 1.11057816e+03,
       3.83897842e+01, 7.01263603e+00, 7.63662712e-01, 4.30491183e-01,
       1.30000000e+01, 6.24116001e-03, 2.54386183e+02, 3.22101079e+02,
       6.30000000e+02, 2.55000000e+01, 1.00500000e+03, 4.86076971e+00,
       1.61001160e+00, 5.00000000e+00, 1.30000000e+01, 7.00000000e+00,
       4.87077535e-02])

In [11]:
estimator = RandomForestClassifier(n_estimators=100)
estimator.fit(X_train, y_train)
pd.DataFrame([{'feature': k, 'importance': v} for k, v in sorted(zip(feature_names, estimator.feature_importances_), key=lambda x: -x[1])])

,feature,importance
0,avg(deg | deg > avg_deg),0.141215
1,avg_deg,0.131186
2,max_deg,0.102778
3,std_deg,0.098875
4,median_deg,0.083914
5,n,0.052073
6,log(n/k * p_in/p_out),0.046110
7,n/k * p_in/p_out,0.042857
8,n/k,0.025500
9,max_clique/(n/k),0.022975


In [12]:
estimator = RandomForestClassifier(n_estimators=100)
selector = RFE(estimator, n_features_to_select=2, verbose=1)
selector = selector.fit(X_train, y_train)
print(list(zip(feature_names, selector.support_)))
print(list(zip(feature_names, selector.ranking_)))

Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
[('n', False), ('k', False), ('p_in', False), ('p_out', False), ('n/k', False), ('p_in/p_out', False), ('log(n)/k * p_in/p_out', False), ('n/k * p_in/p_out', False), ('log(n/k) * 

In [13]:
pd.DataFrame(zip(feature_names, selector.support_, selector.ranking_), columns=['feature', 'to choose', 'rank']).sort_values('rank')

,feature,to choose,rank
16,avg(deg | deg > avg_deg),True,1
15,std_deg,True,1
14,avg_deg,False,2
18,max_deg,False,3
0,n,False,4
17,median_deg,False,5
7,n/k * p_in/p_out,False,6
9,log(n/k * p_in/p_out),False,7
2,p_in,False,8
23,max_clique,False,9


In [14]:
estimator = RandomForestClassifier(n_estimators=100)
estimator.fit(X_train[:, selector.support_], y_train)

RandomForestClassifier()

In [15]:
print('\n'.join([f'{k}\t{v:.3f}' for k, v in sorted(zip(np.array(feature_names)[selector.support_], estimator.feature_importances_), key=lambda x: -x[1])]))

avg(deg | deg > avg_deg)	0.530
std_deg	0.470


In [16]:
y_pred = estimator.predict(X_val[:, selector.support_])

In [17]:
accuracy_score(y_val.ravel(), y_pred.ravel())

0.9475

In [18]:
f1_score(y_val.ravel(), y_pred.ravel())

0.7675276752767528